<a href="https://colab.research.google.com/github/dakyommii/study/blob/main/gan_atoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import random
import os
from os import listdir
from os.path import isfile, join
import PIL
from PIL import Image
from glob import glob

import IPython.display as display

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [15]:
MODEL_SAVE_PATH = os.path.join("/content/drive/My Drive/results")

In [6]:
DATA_PATH = "/content/drive/My Drive/atoi_gan_fin" 

In [4]:
#경로연결
import sys
sys.path.insert(0,"/content/stylegan2")

In [6]:
#stylegan2 파일 다운
!git clone https://github.com/NVlabs/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 131 (delta 0), reused 1 (delta 0), pack-reused 127
Receiving objects: 100% (131/131), 593.76 KiB | 6.25 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [7]:
#stylegan2 사용해 image augmentation 
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
# !pip install ninja

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 125, done.
remote: Total 125 (delta 0), reused 0 (delta 0), pack-reused 125
Receiving objects: 100% (125/125), 1.12 MiB | 19.12 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [8]:
!ls /content/stylegan2

dataset_tool.py  LICENSE.txt		 README.md	   run_training.py
dnnlib		 metrics		 run_generator.py  test_nvcc.cu
Dockerfile	 pretrained_networks.py  run_metrics.py    training
docs		 projector.py		 run_projector.py


In [62]:
!python /content/stylegan2/run_generator.py generate-images \
    --network=gdrive:networks/stylegan2-car-config-b.pkl \
  --seeds=6600-6625 --truncation-psi=0.5

  ##car-config 부분을 캐릭터 데이터로 변경 필요

usage: run_generator.py [-h] {generate-images,style-mixing-example} ...
run_generator.py: error: unrecognized arguments: Drive/results


In [9]:
!mkdir out

In [10]:
#렌덤 시드 설정 함수
def seeder(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

In [11]:
#이미지 생성 함수
import argparse
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys

import pretrained_networks

def generate_images(Gs, seeds, truncation_psi):
   #노이즈 생성
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() \
                  if name.startswith('noise')]
    #이미지 형변환
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if truncation_psi is not None:
        Gs_kwargs.truncation_psi = truncation_psi

#생성된 이미지 저장
    for seed_idx, seed in enumerate(seeds):
        print('Generating image for seed %d/%d ...' % (seed_idx, len(seeds)))
        rnd = np.random.RandomState()
        tflib.set_vars({var: rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(seed, None, **Gs_kwargs) # [minibatch, height, width, channel]
        path = f"/content/drive/My Drive/results{seed_idx}.png"
        PIL.Image.fromarray(images[0], 'RGB').save(path)

In [12]:
sys.path.insert(0, "/content/stylegan2-ada-pytorch")

In [13]:
import torch
import legacy

def expand_seed(seeds, vector_size):
  result = []

  for seed in seeds:
    rnd = np.random.RandomState(seed)
    result.append( rnd.randn(1, vector_size) ) 
  return result

#URL = "https://github.com/jeffheaton/pretrained-gan-fish/releases/download/1.0.0/fish-gan-2020-12-09.pkl"
#URL = "https://github.com/jeffheaton/pretrained-merry-gan-mas/releases/download/v1/christmas-gan-2020-12-03.pkl"
URL = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl"

print(f'Loading networks from "{URL}"...')
device = torch.device('cuda')
with dnnlib.util.open_url(URL) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

vector_size = G.z_dim
# range(8192,8300)
seeds = expand_seed( [8192+1,8192+9], vector_size)
#generate_images(Gs, seeds,truncation_psi=0.5)
print(seeds[0].shape)

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl"...
(1, 512)


In [18]:
sc = dnnlib.SubmitConfig()
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content"#/drive/My Drive/projects/stylegan2"
sc.run_desc = 'generate-images'
# network_pkl = 'gdrive:networks/stylegan2-car-config-a.pkl'
print('Loading networks from "%s"...' % MODEL_SAVE_PATH)
 _G, _D, Gs = pretrained_networks.load_networks(MODEL_SAVE_PATH)
vector_size = Gs.input_shape[1:][0]
seeds = expand_seed(range(8000,8020), vector_size)

Loading networks from "/content/drive/My Drive/results"...


ValueError: ignored

In [55]:
generate_images(Gs, seeds,truncation_psi=0.5)

Generating image for seed 0/20 ...
Generating image for seed 1/20 ...
Generating image for seed 2/20 ...
Generating image for seed 3/20 ...
Generating image for seed 4/20 ...
Generating image for seed 5/20 ...
Generating image for seed 6/20 ...
Generating image for seed 7/20 ...
Generating image for seed 8/20 ...
Generating image for seed 9/20 ...
Generating image for seed 10/20 ...
Generating image for seed 11/20 ...
Generating image for seed 12/20 ...
Generating image for seed 13/20 ...
Generating image for seed 14/20 ...
Generating image for seed 15/20 ...
Generating image for seed 16/20 ...
Generating image for seed 17/20 ...
Generating image for seed 18/20 ...
Generating image for seed 19/20 ...


In [56]:
sc.num_gpus = 1
sc.submit_target = dnnlib.SubmitTarget.LOCAL
sc.local.do_not_copy_source_files = True
sc.run_dir_root = "/content/out/stylegan2"
sc.run_desc = 'generate-images'
network_pkl = 'gdrive:networks/stylegan2-car-config-a.pkl'

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
vector_size = Gs.input_shape[1:][0]
# range(8192,8300)
seeds = seeder( [11,875,4754,7770,8880], vector_size)
#generate_images(Gs, seeds,truncation_psi=0.5)
print(seeds[0].shape)

Loading networks from "gdrive:networks/stylegan2-car-config-a.pkl"...
(1, 512)


In [57]:
STEPS = 500
diff = seeds[4] - seeds[0]
step = diff / STEPS
current = seeds[0].copy()

seeds2 = []
for i in range(STEPS):
  seeds2.append(current)
  current = current + step

generate_images(Gs, seeds2,truncation_psi=0.5)

Generating image for seed 460/500 ...
Generating image for seed 461/500 ...
Generating image for seed 462/500 ...
Generating image for seed 463/500 ...
Generating image for seed 464/500 ...
Generating image for seed 465/500 ...
Generating image for seed 466/500 ...
Generating image for seed 467/500 ...
Generating image for seed 468/500 ...
Generating image for seed 469/500 ...
Generating image for seed 470/500 ...
Generating image for seed 471/500 ...
Generating image for seed 472/500 ...
Generating image for seed 473/500 ...
Generating image for seed 474/500 ...
Generating image for seed 475/500 ...
Generating image for seed 476/500 ...
Generating image for seed 477/500 ...
Generating image for seed 478/500 ...
Generating image for seed 479/500 ...
Generating image for seed 480/500 ...
Generating image for seed 481/500 ...
Generating image for seed 482/500 ...
Generating image for seed 483/500 ...
Generating image for seed 484/500 ...
Generating image for seed 485/500 ...
Generating i

In [15]:
#from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255)
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 15,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   brightness_range = (0.2, 1.0),
                                   shear_range = 0.5,
                                   horizontal_flip = True,
                                   fill_mode='nearest'
                                   )

In [18]:
file_name_in_dir = []

for root,dirs,files in os.walk(DATA_PATH):
  for fname in files:
    full_fname=os.path.join(root,fname)
    file_name_in_dir.append(full_fname)

for file_image in file_name_in_dir:
  img = load_img(file_image)
  x=img_to_array(img)
  x=x.reshape((1,)+x.shape)

  i=0
  for batch in train_datagen.flow(x,save_to_dir=MODEL_SAVE_PATH,save_prefix='characters', save_format='png'):
    i+=1
    if i>5:
      break

FileNotFoundError: ignored